# MARZO 2022

In [1]:
import numpy as np
import pandas as pd
import zipfile
from zipfile import ZipFile
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

## TABLERO SIMETRIK 

In [2]:
tab_simetrik = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\TOPUPS\2022\03. MARZO\TABLEROS - SIMETRIK\Conciliado*.csv', recursive=False)
tab_simetrik

['C:\\Users\\Cristian.Aparicio\\Documents\\TOPUPS\\2022\\03. MARZO\\TABLEROS - SIMETRIK\\Conciliado_marzo_2022.csv']

In [3]:
info_conc = pd.concat([pd.read_csv(f,dtype={'ORDER_ID':object,'PHONE_NUMBER':object,'TRANSACTIONID':object,
                                             'BIN':object,'ULTIMOS 4':object,'NUMAUTH':object})
                       .assign(file_name=os.path.basename(f)) for f in tab_simetrik])
info_conc.columns = info_conc.columns.str.strip().str.lower()
info_conc['fecha clip']= pd.to_datetime(info_conc['fecha clip'])
info_conc['serti_date']= pd.to_datetime(info_conc['serti_date'])
info_conc.sort_values(by=['serti_date'],ascending=True,inplace=True)
info_conc['serti_month'] = info_conc['serti_date'].dt.strftime('%B')
info_conc['clip_month'] = info_conc['fecha clip'].dt.strftime('%B')
info_conc.head()

,fecha clip,serti_date,aprobada,order_id,phone_number,transactionid,bin,ultimos 4,clip_carrier,id,numauth,comision,estatus,monto,cuenta,comision_calculada,iva_comision,file_name,serti_month,clip_month
266885,2022-03-01,2022-03-01,True,49737418,6861128342,53339376,520416,1894,TELCEL,512c0de6-5e4c-402c-a6b4-5892861f9ced,022752,1.20,Approved,30,1,1.20,0.19,Conciliado_marzo_2022.csv,March,March
260963,2022-03-01,2022-03-01,True,49686242,8717691026,53287939,415231,6394,TELCEL,f4d548d3-d569-435e-83f9-b4ccc62dd44e,052675,4.00,Approved,100,1,4.00,0.64,Conciliado_marzo_2022.csv,March,March
260964,2022-03-01,2022-03-01,True,49686243,7352514325,53287940,415231,2776,TELCEL,3a605f7d-689f-4ecd-9f17-6ba1b95c7cab,052840,2.00,Approved,50,1,2.00,0.32,Conciliado_marzo_2022.csv,March,March
260965,2022-03-01,2022-03-01,True,49686247,5620829503,53287946,551238,5102,ATT,0359be6f-5e9f-44f2-b93f-8ddda18a8972,187379,2.10,Approved,30,1,2.10,0.34,Conciliado_marzo_2022.csv,March,March
260966,2022-03-01,2022-03-01,True,49686255,4641187323,53287954,474174,7990,TELCEL,490e2fb8-f25a-4168-9e35-98bef83b01e0,217476,1.20,Approved,30,1,1.20,0.19,Conciliado_marzo_2022.csv,March,March


In [4]:
info_conc.shape

(266886, 20)

## CONCILIACION SERTI_CLIP

In [5]:
url = r'C:\Users\Cristian.Aparicio\Documents\TOPUPS\2022\03. MARZO\SERTI-CLIP TOPUPS MARZO 2022.xlsx'

In [6]:
conc_manual = pd.ExcelFile(url)
conc_manual

In [7]:
conc_manual.sheet_names

['RESUMEN SERTI',
 'RESUMEN',
 'RESUMEN CARRIER',
 'RESUMEN COMISIONES',
 'DETALLE COMPLETO',
 'FOUND',
 'NOT FOUND',
 'DUPLICADOS_INDEX_TOTAL']

In [8]:
all_sheets = pd.read_excel(url, sheet_name=[5],converters={'bin':str,'panTerminacion': str,'transactionId':str,
                                                           'last4_phone_serti':str,'clip_external_tracking_id':str,
                                                          'serti_transaction_id':str,'status_code':str,'last4_phone':str,
                                                          'response_code':str})

In [9]:
all_sheets.keys()

dict_keys([5])

In [10]:
conc_found = pd.concat(all_sheets).reset_index()
conc_found.columns = conc_found.columns.str.strip().str.lower()
conc_found.head()

,level_0,level_1,index,bin,panterminacion,importe,date,transactionid,entrymode,numauth,...,serti_transaction_id,created_at,updated_at,status_code,status_msg,amount,last4_phone,carrier_y,type,response_code
0,5,0,166,527333,8895,100,2022-03-01 00:01:24.236,53231015,Chip,35500,...,53231015,2022-03-01,2022-03-01,1,Approved,100,9911,ATT,emv_sale,00
1,5,1,167,557910,2199,100,2022-03-01 00:05:46.504,53231067,Chip,54643,...,53231067,2022-03-01,2022-03-01,1,Approved,100,7445,TELCEL,emv_sale,00
2,5,2,168,404360,1405,200,2022-03-01 00:06:11.390,53231073,Chip,35600,...,53231073,2022-03-01,2022-03-01,1,Approved,200,0036,TELCEL,emv_sale,00
3,5,3,169,416916,9388,30,2022-03-01 00:06:21.658,53231076,Chip,144307,...,53231076,2022-03-01,2022-03-01,1,Approved,30,3445,TELCEL,emv_sale,00
4,5,4,170,557910,2199,100,2022-03-01 00:06:25.339,53231077,Chip,62554,...,53231077,2022-03-01,2022-03-01,1,Approved,100,7081,TELCEL,emv_sale,00


In [11]:
conc_found.shape

(266845, 44)

In [12]:
conc_found.columns

Index(['level_0', 'level_1', 'index', 'bin', 'panterminacion', 'importe',
       'date', 'transactionid', 'entrymode', 'numauth', 'terminal', 'caja',
       'aprobada', 'claverechazo', 'merchant', 'producto', 'tarjetahabiente',
       'comision', 'ivacomision', 'carrier_x', 'phone', 'folio', 'file_name',
       'last4_phone_serti', 'serti_date', 'index_total', 'index_comisiones',
       'estatus', 'porcentaje_comision_final', 'comision_db',
       'iva_comision_db', 'topup_id', 'transaction_id',
       'clip_external_tracking_id', 'serti_transaction_id', 'created_at',
       'updated_at', 'status_code', 'status_msg', 'amount', 'last4_phone',
       'carrier_y', 'type', 'response_code'],
      dtype='object')

## CRUCE 1: TABLEROS SIMETRIK VS CONCILIACIÓN SERTI_CLIP

In [13]:
cruce_1 = info_conc.merge(conc_found[['transactionid','serti_date','importe']]
                      ,on='transactionid',how='left')
cruce_1['serti_date_y'].fillna('not_found',inplace=True)
cruce_1['importe'].fillna(0,inplace=True)
cruce_1.head()

,fecha clip,serti_date_x,aprobada,order_id,phone_number,transactionid,bin,ultimos 4,clip_carrier,id,...,estatus,monto,cuenta,comision_calculada,iva_comision,file_name,serti_month,clip_month,serti_date_y,importe
0,2022-03-01,2022-03-01,True,49737418,6861128342,53339376,520416,1894,TELCEL,512c0de6-5e4c-402c-a6b4-5892861f9ced,...,Approved,30,1,1.20,0.19,Conciliado_marzo_2022.csv,March,March,2022-03-01 00:00:00,30.00
1,2022-03-01,2022-03-01,True,49686242,8717691026,53287939,415231,6394,TELCEL,f4d548d3-d569-435e-83f9-b4ccc62dd44e,...,Approved,100,1,4.00,0.64,Conciliado_marzo_2022.csv,March,March,2022-03-01 00:00:00,100.00
2,2022-03-01,2022-03-01,True,49686243,7352514325,53287940,415231,2776,TELCEL,3a605f7d-689f-4ecd-9f17-6ba1b95c7cab,...,Approved,50,1,2.00,0.32,Conciliado_marzo_2022.csv,March,March,2022-03-01 00:00:00,50.00
3,2022-03-01,2022-03-01,True,49686247,5620829503,53287946,551238,5102,ATT,0359be6f-5e9f-44f2-b93f-8ddda18a8972,...,Approved,30,1,2.10,0.34,Conciliado_marzo_2022.csv,March,March,2022-03-01 00:00:00,30.00
4,2022-03-01,2022-03-01,True,49686255,4641187323,53287954,474174,7990,TELCEL,490e2fb8-f25a-4168-9e35-98bef83b01e0,...,Approved,30,1,1.20,0.19,Conciliado_marzo_2022.csv,March,March,2022-03-01 00:00:00,30.00


In [14]:
cruce_1.shape

(266886, 22)

## VALIDACIÓN DE FECHAS

In [16]:
cruce_1['valid_fechas'] = cruce_1['serti_date_x'] == cruce_1['serti_date_y']
cruce_1.head()

,fecha clip,serti_date_x,aprobada,order_id,phone_number,transactionid,bin,ultimos 4,clip_carrier,id,...,monto,cuenta,comision_calculada,iva_comision,file_name,serti_month,clip_month,serti_date_y,importe,valid_fechas
0,2022-03-01,2022-03-01,True,49737418,6861128342,53339376,520416,1894,TELCEL,512c0de6-5e4c-402c-a6b4-5892861f9ced,...,30,1,1.20,0.19,Conciliado_marzo_2022.csv,March,March,2022-03-01 00:00:00,30.00,True
1,2022-03-01,2022-03-01,True,49686242,8717691026,53287939,415231,6394,TELCEL,f4d548d3-d569-435e-83f9-b4ccc62dd44e,...,100,1,4.00,0.64,Conciliado_marzo_2022.csv,March,March,2022-03-01 00:00:00,100.00,True
2,2022-03-01,2022-03-01,True,49686243,7352514325,53287940,415231,2776,TELCEL,3a605f7d-689f-4ecd-9f17-6ba1b95c7cab,...,50,1,2.00,0.32,Conciliado_marzo_2022.csv,March,March,2022-03-01 00:00:00,50.00,True
3,2022-03-01,2022-03-01,True,49686247,5620829503,53287946,551238,5102,ATT,0359be6f-5e9f-44f2-b93f-8ddda18a8972,...,30,1,2.10,0.34,Conciliado_marzo_2022.csv,March,March,2022-03-01 00:00:00,30.00,True
4,2022-03-01,2022-03-01,True,49686255,4641187323,53287954,474174,7990,TELCEL,490e2fb8-f25a-4168-9e35-98bef83b01e0,...,30,1,1.20,0.19,Conciliado_marzo_2022.csv,March,March,2022-03-01 00:00:00,30.00,True


In [17]:
cruce_1.shape

(266886, 23)

In [19]:
cruce_1_fecha_distinta = cruce_1[(cruce_1['valid_fechas'] == False)]
cruce_1_fecha_distinta.head()

,fecha clip,serti_date_x,aprobada,order_id,phone_number,transactionid,bin,ultimos 4,clip_carrier,id,...,monto,cuenta,comision_calculada,iva_comision,file_name,serti_month,clip_month,serti_date_y,importe,valid_fechas
2772,2022-03-01,2022-03-01,True,49623128,6693328590,53225489,542878,0792,TELCEL,2b89d195-de22-43f3-9c5d-65eba9948d21,...,200,1,8.00,1.28,Conciliado_marzo_2022.csv,March,March,not_found,0.00,False
2773,2022-03-01,2022-03-01,True,49623194,6692630550,53225553,416916,1255,TELCEL,190641d6-b09c-46d0-8e33-c0bc93135fee,...,30,1,1.20,0.19,Conciliado_marzo_2022.csv,March,March,not_found,0.00,False
2774,2022-03-01,2022-03-01,True,49623199,6624152411,53225558,557910,3963,TELCEL,c6028af3-d4a3-4423-a128-4a58e1025f9c,...,100,1,4.00,0.64,Conciliado_marzo_2022.csv,March,March,not_found,0.00,False
2775,2022-03-01,2022-03-01,True,49623317,3335776797,53225667,547046,7246,UNEFON,b55ab147-45b5-49bf-8b11-3db9eeaba5f4,...,100,1,7.00,1.12,Conciliado_marzo_2022.csv,March,March,not_found,0.00,False
2776,2022-03-01,2022-03-01,True,49623335,9983518658,53225685,416916,7627,TELCEL,e90245a2-c4aa-40a1-83d5-e57195e09408,...,100,1,4.00,0.64,Conciliado_marzo_2022.csv,March,March,not_found,0.00,False


In [20]:
cruce_1_fecha_distinta.shape

(966, 23)

In [21]:
cruce_1_fecha_distinta.to_csv('fecha_distinta.csv')

## NO ENCONTRADOS

In [23]:
cruce_1_not_found = cruce_1[(cruce_1['serti_date_y'] == 'not_found')]
cruce_1_not_found.head()

,fecha clip,serti_date_x,aprobada,order_id,phone_number,transactionid,bin,ultimos 4,clip_carrier,id,...,estatus,monto,cuenta,comision_calculada,iva_comision,file_name,serti_month,clip_month,serti_date_y,importe
2772,2022-03-01,2022-03-01,True,49623128,6693328590,53225489,542878,0792,TELCEL,2b89d195-de22-43f3-9c5d-65eba9948d21,...,Approved,200,1,8.00,1.28,Conciliado_marzo_2022.csv,March,March,not_found,0.00
2773,2022-03-01,2022-03-01,True,49623194,6692630550,53225553,416916,1255,TELCEL,190641d6-b09c-46d0-8e33-c0bc93135fee,...,Approved,30,1,1.20,0.19,Conciliado_marzo_2022.csv,March,March,not_found,0.00
2774,2022-03-01,2022-03-01,True,49623199,6624152411,53225558,557910,3963,TELCEL,c6028af3-d4a3-4423-a128-4a58e1025f9c,...,Approved,100,1,4.00,0.64,Conciliado_marzo_2022.csv,March,March,not_found,0.00
2775,2022-03-01,2022-03-01,True,49623317,3335776797,53225667,547046,7246,UNEFON,b55ab147-45b5-49bf-8b11-3db9eeaba5f4,...,Approved,100,1,7.00,1.12,Conciliado_marzo_2022.csv,March,March,not_found,0.00
2776,2022-03-01,2022-03-01,True,49623335,9983518658,53225685,416916,7627,TELCEL,e90245a2-c4aa-40a1-83d5-e57195e09408,...,Approved,100,1,4.00,0.64,Conciliado_marzo_2022.csv,March,March,not_found,0.00


In [24]:
cruce_1_not_found.shape

(140, 22)

In [25]:
cruce_1_not_found.to_csv('not_found.csv')